# 冻手实验室3.18-NEU校园卡查询中心模拟登陆与数据获取

## 1.本次将学到

- 如何利用浏览器的开发者功能来了解一个页面
- 用与发送http请求的python库-[requests](http://docs.python-requests.org/zh_CN/latest/user/quickstart.html)
- 用与解析网页的python库-[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.zh.html)

## 2.环境配置

**强烈建议大家提前配置好环境**
- python3
- python包：
    - requests   安装方法：`pip install requests`  
    - BeautifulSoup [安装方法见此处](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.zh.html#id5)
    - jupyter notebook  [安装方法见此处](http://jupyter.org/install)
    - matplotlib 安装方法：`pip install -U matplotlib`  或者查看[官方教程](https://matplotlib.org/users/installing.html)
- **推荐使用**chrome或者Firefox浏览器


## 3.导入需要用到的包

In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## 4.具体步骤

- 4.1模拟登录
    - 4.1.1获取登陆页面
    - 4.1.2获取验证码
    - 4.1.3登录

- 4.2数据查询与获取
    - 4.2.1 获取查询页面
    - 4.2.2 进行查询
    - 4.2.3 结果翻页





### 4.1.1获取登陆页面

这里，我们先获取尝试利用request来发送get请求登录页面。

为了保持登录的状态，我们需要用到requests的session对象，他可以自动地为我们保存cookies。而一般网站会利用cookies来保存用户的状态。

我们还将使用BeautifulSoup来解析此页面

In [ ]:
session = requests.Session()

In [ ]:
url_login='http://ecard.neu.edu.cn/SelfSearch/Login.aspx'
resp_login_page = session.get(url=url_login)

In [ ]:
#查看获取到网页内容
print(resp_login_page.text)

以上内容，我们可以用正则表达式来找出我们想要的信息，不过太难了。让我们来尝尝神奇的BeautifulSoup😎

第一个参数是网页的内容，第二个参数是所用的解析器，‘html.parser’是python标准库中自带的。

这样我们就获得了一个文档对象。

In [ ]:
soup_login_page = BeautifulSoup(resp_login_page.text, 'html.parser')

通过这个文档对象可以轻易地获取我们想获取的内容。

以下代码找到了页面中的title标签。

关于find方法的更多介绍，你可以[看这里](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.zh.html#find)

- find返回第一个结果。
- find_all方法会返回一个储存了全部结果的list

In [ ]:
soup_login_page.find('title')

也可以直接用id来搜索

In [ ]:
soup_login_page.find(id='one1')

那你来试试看，获取VIEWSTATE和EVENTVALIDATION的值。

提示：他们的值写的html里面，并且有特定的id。
标签的属性，可以通过`xxx['abc']`来获取

In [ ]:
VIEWSTATE= soup_login_page.find(id = '__VIEWSTATE')['value']
EVENTVALIDATION= soup_login_page.find(id = '__EVENTVALIDATION')['value']
print(VIEWSTATE+'\n')
print(EVENTVALIDATION)

### 4.1.2获取验证码
通常你看到的网页中的图片，在html中只是写了一个图片的链接。
我们的验证码也需要从一个图片获得。


In [ ]:
# 获得验证码的url，你需要写一行,提示：你可能需要随机数函数
url_captcha= 'http://ecard.neu.edu.cn/SelfSearch/validateimage.ashx?' + str(random.random())
print(url_captcha)
#向该链接发送get请求，并获取Response对象，你需要写一行
resp_captcha = session.get(url=url_captcha)

将图片保存。
[关于二进制的响应内容](http://docs.python-requests.org/zh_CN/latest/user/quickstart.html#id4)

In [ ]:
with open('captcha.gif', 'wb') as f:
    f.write(resp_captcha.content)
    f.close()

读取图片文件，并显示。

[教程看这里](https://matplotlib.org/tutorials/introductory/images.html#sphx-glr-tutorials-introductory-images-py)

In [ ]:
img = mpimg.imread('captcha.gif')
plt.imshow(img)

In [ ]:
#把看到的验证码填到这里,注意是字符串形式哟
captcha= '5115'

### 4.1.3登录

登录需要向服务器发送一个post请求，
你需要先获取所需的链接和post请求的请求体内容

In [ ]:
userName = ''
passwd= ''

url_login = 'http://ecard.neu.edu.cn/SelfSearch/Login.aspx'
postdata = {
    '__EVENTVALIDATION': EVENTVALIDATION,
    '__VIEWSTATE': VIEWSTATE,
    '__EVENTTARGET': 'btnLogin',
    'txtUserName': userName,
    'txtPassword': passwd,
    'txtVaildateCode': captcha,
    'hfIsManager':0
}

In [ ]:
loginresponse = session.post(url=url_login, data=postdata)
print('跳转的链接：', loginresponse.url)

如果返回的是'http://ecard.neu.edu.cn/SelfSearch/Index.aspx' 则说明我们已经登录成功并且跳转到了登陆后的页面。

来看看你的证件照吧！

In [ ]:
# 自己去找照片的url
url_profile_photo='http://ecard.neu.edu.cn/SelfSearch/User/Photo.ashx'

#发送get请求
resp_profile_photo = session.get(url=url_profile_photo)

#你可以看请求对象的url属性值十分为http://ecard.neu.edu.cn/SelfSearch/User/Photo.ashx 来判断是否是登录成功的。因为要是没成功，回跳转到登录页面
print('跳转链接', resp_profile_photo.url)

#保存图片
with open('userPic.jpg', 'wb') as f:
    f.write(resp_profile_photo.content)
    f.close()


In [ ]:
#查看图片，自己写
img = mpimg.imread('userPic.jpg')
plt.imshow(img)


### 4.2数据查询与获取

### 4.2.1 获取查询页面

4.2.3 结果翻页

In [ ]:
# 自己找消费记录是向哪个url请求的
url_consumeInfo= 'http://ecard.neu.edu.cn/SelfSearch/User/ConsumeInfo.aspx'

#获取页面
consume_response0=session.get(url_consumeInfo)
consume_soup = BeautifulSoup(consume_response0.text, 'html.parser')

#解析出以下两个变量的值
VIEWSTATE= consume_soup.find(id='__VIEWSTATE')['value']
EVENTVALIDATION= consume_soup.find(id='__EVENTVALIDATION')['value']
print(VIEWSTATE)

### 4.2.2 进行查询与数据解析

In [ ]:
#构造postdata
postdata_consume = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__VIEWSTATE': VIEWSTATE,
    '__EVENTVALIDATION': EVENTVALIDATION,
    'ctl00$ContentPlaceHolder1$rbtnType': 0,
    'ctl00$ContentPlaceHolder1$txtStartDate': '2018-03-11',
    'ctl00$ContentPlaceHolder1$txtEndDate': '2018-03-18',
    'ctl00$ContentPlaceHolder1$btnSearch': '查  询',
}

resp_consume_1=session.post(url_consumeInfo,data=postdata_consume)
soup_consume_1 = BeautifulSoup(resp_consume_1.text, 'html.parser')


In [ ]:
#找到显示交易信息的table
list_page = soup_consume_1.find(id='ContentPlaceHolder1_AspNetPager1').find_all('a')
a = int(list_page[len(list_page) - 3].text)

In [ ]:
#想办法利用find_all方法，把每一行的数据提出来，再把一行中的对应内容提取出来
#提示：对于一个文档对象  可以利用abc.text 来获取其文字内容
#提示：关于HTML的table，可以看这里http://www.w3school.com.cn/tags/tag_table.asp




### 4.2.3 翻页（自己完成）

1. 探索翻页实现的机制
2. 构造对应的请求数据实现翻页

In [ ]:
VIEWSTATE= soup_consume_1.find(id='__VIEWSTATE')['value']
EVENTVALIDATION= soup_consume_1.find(id='__EVENTVALIDATION')['value']
postdata_consume2 = {
    '__EVENTTARGET':'ctl00$ContentPlaceHolder1$AspNetPager1',
    '__EVENTARGUMENT':'2',
    '__VIEWSTATE': VIEWSTATE,
    '__EVENTVALIDATION': EVENTVALIDATION,
    'ctl00$ContentPlaceHolder1$rbtnType': 0,
    'ctl00$ContentPlaceHolder1$txtStartDate': '2018-03-11',
    'ctl00$ContentPlaceHolder1$txtEndDate': '2018-03-18',
}
resp_consume_2= session.post(url_consumeInfo,data=postdata_consume2)
soup_consume_2 = BeautifulSoup(resp_consume_2.text, 'html.parser')

In [ ]:
info_table= soup_consume_2.find_all('table')[1]
for line in info_table.find_all('tr'):
    for thh in line.find_all('th'):
        print(thh.text)
    for tdd in line.find_all('td'):
        print(tdd.text)

In [ ]:
import pandas as pd  

In [ ]:
# a = ['one','two','three']  
# b = [1,2,3]  
# save = pd.DataFrame({'english':a,'number':b})  
# save.to_csv('b.txt',index=False,sep=',')  

In [ ]:
info_table= soup_consume_2.find_all('table')[1]
list1 = []
list2 = []
for line in info_table.find_all('tr'):
    for thh in line.find_all('th'):
        list1.append(thh.text)
    for tdd in line.find_all('td'):
        list2.append(tdd.text)

In [ ]:
list1[0] = list1[0].strip('\n')

In [ ]:
time = []
detail = []
money = []
balance = []
operator = []
workstation = []
terminal = []   
for i in range(0,10):
    list2[i*7] = list2[i*7].strip('\n')
    time.append(list2[i*7])
    detail.append(list2[i*7+1])
    money.append(list2[i*7+2])
    balance.append(list2[i*7+3])
    operator.append(list2[i*7+4])
    workstation.append(list2[i*7+5])
    terminal.append(list2[i*7+6])

In [ ]:
data = {list1[0]:time,list1[1]:detail,list1[2]:money,list1[3]:balance,list1[4]:operator,list1[5]:workstation,list1[6]:terminal}
frame = pd.DataFrame(data)
frame.to_csv('result.txt',index=False,sep=',')  
frame